# Project 1:  Risk Preferences and Elicitation
# GRADING: DOES IT WORK? GOOD DOCUMENTATION? GOOD NAMES FOR VARIABLES? 

A typical project has three stages not necessarily done in any specific order.

1.  Plan your project.  This will be done in this notebook. 
2.  Set up your editor (virtual-Studio) code, create a virtual environment, and do an initial commit to your GitHub repository.
3.  Test your project.  Here we will talk about three kinds of tests.  Code tests, logging, and user API.  All of this will be done in a testing notebook and visual studio code.     

In this project, we will use a procedural programming approach to build a `risk` module to model and elicit risk preferences.  This project is broken down into some functions which are described in the steps described below:

* Step 1:  Code `utility functions` defined on lottery outcomes.  
* Step 2:  Decide on, and code, a general `lottery data structure`.
* Step 3:  Code an `expected value function` and `expected utility function`.
* Step 4:  Code `complementary functions` for risk preferences. 
* Step 5:  Code a `lottery choice function`.
* Step 6:  Code the `Holt-Laury procedure` for risk preference elicitation.
* Step 7:  Code the `step-wise elicitation algorithm`.
* Step 8:  Build a `risk preference module` called `risk`.

## Step One:  Code utility functions

Research the literature starting [here](https://en.wikipedia.org/wiki/Risk_aversion) and find three additional utility functions that you can use to model risk preferences.  Build these utility functaions as python functions in the cells below.  I will start you out with a simple one. 

In [66]:
def linear_utility(m, a=5, b=.75):
    """linear utility of money
    
        args: m, float, amount of money.
              a, float, intercept.
              b, float, slope.
              
        returns: util, float, utility of money m.    
    """
    util = a + b*m
    return util

In [67]:
# test functions

m1 = 0
m2 = 10
p1 = .5
p2 = .5

# test linear_utility function
print(linear_utility(m2))
print(f"expected value = {p1*m1 + p2*m2}, expected utility = {p1*linear_utility(m1) + p2*linear_utility(m2)}")

12.5
expected value = 5.0, expected utility = 8.75


# Additional utility functions

1. CARA (Constant Absolute Risk Aversion)
2. CRRA (Constant Relative Risk Aversion)
3. Quadratic utility

In [68]:
import math
def cara(m, a=0.5):
    """Exponential Utility Constant Absolute Risk Aversion (CARA): u(m) = 1 - exp(-a*m)

        args:
            m(float): amount of money (payoff)
            a(float): >0, absolute risk aversion parameter (larger a, more risk averse)
        
        returns:
            util: float, utility of money m

        raises:
            ValueError: if a <= 0
    """
    if a <=0:
        raise ValueError("Parameter 'a' must be > 0.")
    return 1 - math.exp(-a * m) #this is returning the utility of money
while True:
    try:
        m_val = float(input("Enter m (Money/Payoff): "))
        a_val = float(input("Enter a (> 0, risk-aversion parameter): "))
        u = cara(m_val, a=a_val)
        print(f"u(m) = {u}")
        break
    except ValueError as e:
        print(f"Input error: {e}. Please try again.\n")

Input error: could not convert string to float: ''. Please try again.

Input error: could not convert string to float: ''. Please try again.

u(m) = 0.9999999999861121


In [69]:
import math

def crra(m, gamma=2.0):
    """Power Utility (CRRA). 

        u(m) = (m^(1-gamma)-1) / (1 - gamma)        if gamma != 1
            = ln(m)                                if gamma ==1  
        args:
            m (float): > 0, money/wealth
            gamma (float): > 0, coefficient of relative risk aversion (larger gamma, more risk averse)

        returns: 
            util (float): utility of money m
        
        raises:
            ValueError: if m <= 0 or gamma <= 0
    """
    if m <= 0:
        raise ValueError("Parameter 'm' must be > 0 for CRRA utility.")
    if gamma <= 0:
        raise ValueError("Parameter 'gamma must be > 0.")
    if gamma == 1:
        return math.log(m)
    return (m**(1 - gamma) - 1) / (1 - gamma)
while True:
    try:
        m_val = float(input("Enter m (> 0, money/wealth): "))
        gamma_val = float(input("Enter gamma (> 0, relative risk aversion): "))
        u = crra(m_val, gamma=gamma_val)
        print(f"u(m) = {u}")
        break
    except ValueError as e:
        print(f"Input error: {e}. Please try again.\n")

u(m) = 0.2496


In [ ]:
def quadratic(m, a=1.0, b=0.1):
    """Quadratic utility: u(m) = a*m - 0.5*b*m^2
    
    args:
        m (float): money/payoff
        a (float): slope at zero wealth
        b (float): > 0, curvature parameter (keeps the function concave)
        
    returns:
        util (float): utility of money m

    raises:
        ValueError: if b <= 0 (would break concavity)
        ValueError: if a - b*m <= 0 (would make marginal utility non-positive)
    """

    if b <= 0:
        raise ValueError("Parameter 'b' must be > 0 to keep utility concave.")
    if a - b*m <= 0:
        raise ValueError("Inputs result in non-positive marginal utility (a - b*m <= 0). Choose smaller m or larger a, or smaller b.")
    util = a*m - 0.5*b*(m**2)
    return util

while True:
    try:
        m_val = float(input("Enter m (money/payoff): "))
        a_val = float(input("Enter a (baseline slope): "))
        b_val = float(input("Enter b (> 0, curvature): "))

        u = quadratic(m_val, a=a_val, b=b_val)
        print(f"u(m) = {u}")
        break
    except ValueError as e:
        print(f"Input error: {e}. Please try again. \n")

Input error: Inputs result in non-positive marginal utility (a - b*m <= 0). Choose smaller m or larger a, or smaller b.. Please try again. 

Input error: Inputs result in non-positive marginal utility (a - b*m <= 0). Choose smaller m or larger a, or smaller b.. Please try again. 



> You can think about additional utility functions you might want to include in this module.

## Step Two: Design and code lottery data structure  

The simplest version of a lottery is a list of dictioaries, where each dictionary indicates an outcome and the probability, greater than or equal to 0, of that outcome happening.  Here is an example:

```python
lottery = [{'out':0, 'prob':0.5}, {'out':10, 'prob':0.5}]
```
Obviously the list can have as many dictioaries as we would like.  Lotteries must have the following additional property.  The sum of the probabilities accross the elements (or dictioaries) of the lottery must equal 1.0.  In addition the value associate with 'out' can be either a payoff as in the example or another lottery.  When this happens we call this a compound lottery.  Here is an example.

```python
compound_lottery = [{'out':lottery, 'prob':0.5}, {'out':10, 'prob':0.5}]
```
The first step in any lottery choice problem is to build the lotteries.  Lets define two functions to do this.

In [ ]:
def input_lottery():
    """Builds a lottery from user input
    
        returns:  lottery, a list of dictionaries
    """
    tol = 1e-9 #small tolerance I got from class to deal with rounding error
    lottery = []

#take out any "for I in range". Professor doesnt like that it doesnt know what im indexing through
    while True:
        try:
            n = int(input("How many outcomes in this lottery? (integer >= 1): "))
            if n >= 1:
                break
            else:
                print("Please enter an integer >= 1.\n")
        except ValueError as e:
            print(f"Input error: {e}. Please try again.\n")
    remaining = 1.0
    for i in range(1, n+1):
        while True:
            t = input(f"Outcome {i}: type 'n' for numeric payoff, 'l' for nested lottery: ").strip().lower()
            if t in ("n", "l"):
                break
            print("Please type 'n' or 'l'.\n")
        if t == "n":
            while True:
                try:
                    out_val = float(input(f"Enter payoff for outcome {i}: "))
                    break
                except ValueError as e:
                    print(f"Input error: {e}. Please try again.\n")
        else:
            print(f"Building nested (compound) lottery for outcome {i}...")
            out_val = input_lottery()
        if i < n:
             while True:
                try:
                    prob_val = float(input(f"Enter probability for outcome {i} (remaining {remaining:.6f}): "))
                    if prob_val < 0:
                        print("Probability must be >= 0.\n")
                    elif prob_val > remaining + tol:
                        print(f"Probability cannot exceed the remaining {remaining:.6f}.\n")
                    else:
                        remaining -= prob_val
                        break
                except ValueError as e:
                    print(f"Input error: {e}. Please try again.\n")
        else:
            prob_val = remaining
            if prob_val < -tol or prob_val > 1 + tol:
                raise ValueError("Probabilities do not sum to 1. Please restart the lottery input.")
            prob_val = max(0.0, min(1.0, prob_val))
            print(f"Auto-setting probability for outcome {i} to the remaining {prob_val:.6f} so total sums to 1.")
        lottery.append({'out': out_val, 'prob': prob_val})
    return lottery
print("Your lottery:", input_lottery())

Building nested (compound) lottery for outcome 2...
Auto-setting probability for outcome 2 to the remaining 0.500000 so total sums to 1.
Auto-setting probability for outcome 3 to the remaining 0.000000 so total sums to 1.
Your lottery: [{'out': 10.0, 'prob': 0.3}, {'out': [{'out': 10.0, 'prob': 0.5}, {'out': 10.0, 'prob': 0.5}], 'prob': 0.7}, {'out': 10.0, 'prob': 0.0}]


In [ ]:
import random
def make_random_lotteries(number=2, max_pay = 100, compound=False, negative=False):
    """Builds a random list of lotteries
    
        Lottery payoffs are random draws beteen min_pay and max_pay,
        Lottery probabilities are random uniform draws that sum to one.
    
        args:
            number = int > 0, number of lotteries in list.
            compound = bool, if True allows compound lotteries one deep,
            negative = bool, if False min_pay = 0, if True min_pay = -max_pay.
    """
    min_pay = -float(max_pay) if negative else 0.0 #min_pay will be the negative of max_pay if it's negative
    max_pay = float(max_pay)
    def _simple_lottery():
        k = random.randint(2, 4) #2-4 possible outcomes
        payoffs = [random.uniform(min_pay, max_pay) for _ in range(k)]
        weights = [random.random() for _ in range(k)]
        total = sum(weights)
        probs = [w / total for w in weights]
        return [{'out': payoffs[i], 'prob': probs[i]} for i in range (k)]
    lotteries = []
    for _ in range(number):
        lot = _simple_lottery()
        if compound:
            j = random.randrange(len(lot))
            lot[j]['out'] = _simple_lottery()
        lotteries.append(lot)
    
    return lotteries
print(make_random_lotteries())
print(make_random_lotteries(number=3, max_pay=50, compound=True, negative=True))

[[{'out': 38.35380300107462, 'prob': 0.14962519252687587}, {'out': 86.95077314585747, 'prob': 0.49340199151066655}, {'out': 46.26951319487264, 'prob': 0.3569728159624576}], [{'out': 27.715403436566977, 'prob': 0.008297567269067715}, {'out': 82.54638157138592, 'prob': 0.15039308461559375}, {'out': 57.04990050669224, 'prob': 0.22584729099074977}, {'out': 40.916301306012095, 'prob': 0.6154620571245887}]]
[[{'out': [{'out': 30.669059580604113, 'prob': 0.4296298748441994}, {'out': -14.715902976968955, 'prob': 0.5016036018079739}, {'out': 22.555452927765955, 'prob': 0.06876652334782667}], 'prob': 0.16388722800817723}, {'out': -32.543081935807734, 'prob': 0.48577048434047676}, {'out': -38.79454438277384, 'prob': 0.35034228765134606}], [{'out': [{'out': -49.97177657861981, 'prob': 0.5019421685853949}, {'out': 25.7870220662752, 'prob': 0.13121894095077707}, {'out': -14.076668326968203, 'prob': 0.3668388904638281}], 'prob': 0.5352476522031248}, {'out': -24.02417765157894, 'prob': 0.4647523477968

## Step Three: Code an expected value function and expected utility function

### Expected Value Coded Here

In [ ]:
def expected_value(lottery):
    """Calculate expected value of a lottery
    
        arg:
            lottery, list of dictioaries
        
        returns:
            ev, float, expected value of the lottery
    """
    ev = 0.0 #starting the ev at 0.0 to add to through the code
    for d in lottery:
        out = d['out']
        p = d['prob']
        if isinstance(out, list): #if (out) is a list, this outcome is a compound or nested lottery
            ev += p * expected_value(out) #getting the expected value of the inner lottery (out) and multilping it by the outcomes probabiliy p. This is the weight added to the running total ev
        else:
            ev += p * float(out) #here (out) is just a number so I made it a float and multiplied by probablility to get weight to add to ev
        
    #TODO Expected Value Calculation
    return ev

In [ ]:
# Text expected value function
lottery = [{'out':0, 'prob':0.5}, {'out':10, 'prob':0.5}]
print(f"expected value = {expected_value(lottery)}")
compound_lottery = [{'out':lottery, 'prob':0.5}, {'out':10, 'prob':0.5}]
print(f"expected value = {expected_value(compound_lottery)}")

expected value = 5.0
expected value = 7.5


In [ ]:
# You can always assign a variable to a function and pass a function to a function.  Here is an example.

def u(m, f):
    return f(m)
    
m = 10
util = linear_utility
print(type(util), util)

print(f"utility of {m} = {u(m, util)}")
print('or')
print(f"utility of {m} = {u(m, linear_utility)}")


<class 'function'> <function linear_utility at 0x11186ed40>
utility of 10 = 12.5
or
utility of 10 = 12.5


### Code Expected Utility Here

In [14]:
def expected_utility(lottery, u):
    """Calculate expected utility of a lottery
    
        arg:
            lottery, list of dictioaries containing 
                     keys, values {'prob': pr, 'outcome': out}
                         pr, float between 0.0 and 1.0 inclusive
                         out, either another lottery or a float payoff
                     
            u, utility function, returns utility of a payoff outcome
        
        returns:
            eu, float, expected utility of the lottery
    """
    eu = 0.0
    for d in lottery:
        p = d['prob']
        out = d.get('out')
        if isinstance(out, list):
            eu += p * expected_utility(out, u) #if (out) is another lottery, this is recursion
        else:
            eu += p * u(float(out))

    # TODO: Insert Code Here
    return eu

In [ ]:
#Test for the expected utility function
lottery = [{'out': 0, 'prob': 0.5}, {'out': 10, 'prob': 0.5}]
print(f"expected utility = {expected_utility(lottery, linear_utility)}")

compound_lottery = [{'out': lottery, 'prob': 0.5}, {'out': 10, 'prob': 0.5}]
print(f"expected utility = {expected_utility(compound_lottery, linear_utility)}")

expected utility = 8.75
expected utility = 10.625


## Step Four: complementary functions

In this step you will add three complementary functions.  The first function reduces a compound lottery to a simple lottery.  The second function calculates the certainty equivalent of a simple lottery.  The third calculates the risk premium of a lottery. 

In [ ]:
def reduce_lottery(lottery):
    """ Reduces compound lottery to a simple lottery.
    
        A compound lottery has sub-lotteries as outcomes.
        A simple lottery only has payoffs as outcomes.
        
        args:
            lottery, list of dictionaries.
        returns:
            simple_lottery, list of dictionaries with no sub-lottery.
    """
    payoff_prob = {}

    def walk(L, weight): #walks down each sub-lottery and multiplies probabilities until it gets to a numerical value instead of another sub-lottery
        for d in L: #for each outcome dictionary in this lottery list
            p = d['prob'] * weight #multipy the probability on each sub branch by the probability that we got already (weight)
            out = d.get('out') #gets the outcome 
            if isinstance(out, list):#if the outcome is another lottery, go deeper, and carry the multiplied probablility 
                walk(out, p)
            else:
                x = float(out) #when the outcome is a number instead of a sub-lottery, convert to float
                payoff_prob[x] = payoff_prob.get(x, 0.0) + p #if the payoff is a number, add the probability to the total
    walk(lottery, 1.0)

    simple_lottery = [{'out': x, 'prob': payoff_prob[x]} for x in sorted(payoff_prob)]
    return simple_lottery

In [ ]:
#example for reduced lottery
inside_lottery = [{'out': 0, 'prob': 0.5}, {'out': 10, 'prob': 0.5}]

outside_lottery = [{'out': inside_lottery, 'prob': 0.6}, {'out': 10, 'prob': 0.4}]

print(reduce_lottery(outside_lottery))

[{'out': 0.0, 'prob': 0.3}, {'out': 10.0, 'prob': 0.7}]


In [ ]:
def certainty_equivalent(lottery, u):
    """ Returns the certainty equivalent (ce) of a lottery.
    
        u(ce) = expected_utility(lottery, u)
        
        args:
            lottery, list of dictionaries.
            u, utility function defined over payoffs.
        returns:
            ce, float, certainty equivalent
    """
    eu = expected_utility(lottery, u) #eu is the probablility weighted average of utilities

    simple = reduce_lottery(lottery) #from the previous code block, gets rid of any sub-lotteries
    low = min(d['out'] for d in simple) #go through all the outcomes and find the minimum
    high = max(d['out'] for d in simple) #go through all the outcomes and find the maximum

    if abs(high-low) < 1e-12: #if the max and min are the same, the certainty equivalent will just be that payoff
        return float(low) #returns the payoff as a float
    
    for _ in range(100):
        mid = 0.5 * (low + high) #finding the middle point between low and high 
        if u(mid) < eu: #if the mid point is less than expected utility, the payoff is too low
            low = mid #move the low up to mid
        else:
            high = mid #this means payoff is too high so we move high down to mid point
        if (high - low) < 1e-9: #if the interval between max and min tests gets small enough, return the midpoint at the certainty equivalent
            break
    return 0.5 * (low + high)

In [ ]:
#example for certainty equivalent
def linear_utility(m, a=5, b=0.75):
    return a + b*m
lottery = [{'out': 0, 'prob': 0.5}, {'out': 10, 'prob': 0.5}]

print(certainty_equivalent(lottery, linear_utility))

4.999999999708962


In [5]:
def risk_premium(lottery, u):
    """ Returns the risk premium (rp) of a lottery.
    
        rp = expected_value(lottery) - certainty_equivalent(lottery, u)
        
        args:
            lottery, list of dictionaries.
            u, utility function defined over payoffs.
        returns:
            rp, float, risk premium
    """
    ev = expected_value(lottery)
    ce = certainty_equivalent(lottery, u)

    rp = ev - ce
    if abs(rp) < tol: 
        rp = 0.0
    return float (rp)

## Step Five:  Lottery choice function

Chooses lottery from list with the highest expected utility.

In [10]:
def lottery_choice(lottery_list, u):
    """Calculate expected utility of a lottery
    
        arg:
            lottery_list, list of lotteries 
            u, utility function, returns utility of a payoff outcome
        
        returns:
            lottery_index, eu  expected utility of the lottery
    """
    eu = 0.0
    lottery_index = None
    # TODO: Insert Code Here

    best_eu = float("-inf") #stores the best EU at negative infinity
    best_idx = None #stores the index of the best lottery

    for i, lot in enumerate(lottery_list):
        eu_i = expected_utility(lot, u) #goes through each lottery and computes the expected utility with expected_utility
        if eu_i > best_eu:
            best_eu = eu_i
            best_idx = i #if the current eu is more than the previous biggest, replace it
    lottery_index = best_idx
    eu = float(best_eu)
    return lottery_index, eu

## Step Six:  The Holt-Laury procedure

This is a well known procedure published by Holt and Laury and avialable on JSTOR [here](https://www.jstor.org/stable/3083270?Search=yes&resultItemClick=true&searchText=Holt&searchText=and&searchText=Laury&searchUri=%2Faction%2FdoAdvancedSearch%3Fc5%3DAND%26amp%3Bc6%3DAND%26amp%3Bf1%3Dall%26amp%3Bc3%3DAND%26amp%3Bq2%3D%26amp%3Bf5%3Dall%26amp%3Bq0%3DHolt%2Band%2BLaury%26amp%3Bf4%3Dall%26amp%3Bc4%3DAND%26amp%3Bsd%3D%26amp%3Bgroup%3Dnone%26amp%3Bq5%3D%26amp%3Bla%3D%26amp%3Bf0%3Dall%26amp%3Bisbn%3D%26amp%3Bf6%3Dall%26amp%3Bacc%3Don%26amp%3Bc1%3DAND%26amp%3Bq4%3D%26amp%3Bf3%3Dall%26amp%3Bq1%3D%26amp%3Bq6%3D%26amp%3Bpt%3D%26amp%3Bc2%3DAND%26amp%3Bf2%3Dall%26amp%3Bq3%3D%26amp%3Bed%3D&refreqid=search%3A63236d1b94e184a59d9526d0f794a141&socuuid=5613aad0-8774-4367-aeab-53105ec2672c&socplat=email).  In this step you code two functions.  The first function makes the list of Holt-Laury lottery choices.  The second function implements the Holt-Laury procedure.  Depending on the value of a default argument the subject will either be a human inputing choices of your `lottery_choice` function.

In [7]:
def build_holt_laury():
    """Returns list of Holt-Laury lottery pairs.
    
        returns:
            holt_laury_lotteries, list of lists
                inner list is a pair of lotteries for choice
    
    """
    high_A, low_A = 6.0, 4.0 #these are the payoffs, safer option A pays 6 with the probability p, else 4
    high_B, low_B = 10.0, 1.0 #same as above but riskier
    
    pairs = [] #starts an empty list
    for k in range(1,11):
        p = k / 10.0 #this is looping through 1-10 to make the probabilities 0.1,0.2...1.0
        option_A = [
            {'prob': p, 'out': high_A},
            {'prob': 1-p, 'out': low_A}, #showing what I said in line 9
        ]
        option_B = [
            {'prob': p, 'out': high_B},
            {'prob': 1-p, 'out': low_B},
        ]
        pairs.append([option_A, option_B])
    return pairs

In [15]:
def holt_laury_choices(lottery_list, u = None):
    """Returns list of lottery choices from lottery_list
    
        If u == None, human makes choices, otherwise
            lottery choice(inner_list, u) makes choices.
    
        args:
            lottery_list = [[lottery, ..., lottery], ..., 
                            [lottery, ..., lottery]]
            u, utility function over payoffs, or None
        returns:
            lottery_list_choices
    
    """
    choices = [] #starts an empty list
    for i, pair in enumerate(lottery_list): #Go through each holt laury pair
        if u is None:
            while True:
                resp = input(f"Row {i+1}: choose 0 (lest) or 1 (right): ").strip()
                if resp in ("0", "1"):
                    choices.append(int(resp)) #asks user to choose which lottery it wants and appends their choice to the list choices
                    break
                else:
                    print("Please enter 0 or 1.")
        else:
            idx, _eu = lottery_choice(pair, u) #uses previous function to choose the higher eu lottery in the row
            choices.append(idx) #append the chosen index
    return choices
lottery_list = build_holt_laury()
def linear_utility(m):
    return m
choices = holt_laury_choices(lottery_list, u=linear_utility)
print(choices)

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


## Step Seven:  Stepwise elicitation algorithm

The stepwise elicitation algorithm will first build a piecewise linear utility function for a human subject. You will then have your piecewise linear function make choices in the Holt-Laury procedure.  

### Algorithm

Step One: Set U(min) = 0 and U(max) = 100.

Step Two:  Ask for ce_1 of lottery = `[min, .5, max, .5]` to find U(ce_1) (see how below)

Step Three: Now ask for ce_2 of `[min, .5, ce_1, .5]` to find U(ce_2)

Step Four:  Now ask for ce_3 of `[ce_1, .5, max, .5]` to find U(ce_3)

Step n+1:  Take any two number x < y, where you know U(x) and U(y) from previous steps, and ask for ce_n of `[x, .5, y, .5]` to find U(ce_n).

Last step:  Given that you now know `[U(min), U(x1), ..., U(xN), U(max)]`, build a piecewise linear function V through these points.

#### How to compute U(ce_n)

This assumes ce_n was requested from a lottery `[x, .5, y, .5]` where U(x) and U(y) are know.    

Since `EU(lottery) = U(ce)`, we now know, `U(ce) = .5*U(x) + .5*U(y)`.

For step two, `U(ce_1) = .5*U(min) + .5*U(max)` = `.5*0 + .5*100 ` = ` 50`.


In [ ]:
def make_piecewise_linear(points):
    #COME BACK TO THIS TO SEE IF ITS NECCESARY

def stepwise_elicit(min_pay, max_pay, ask_ce, num_rounds=3):
    """min_pay, max_pay: endpoints on money axis"""

## Step Eight:  risk module

In this step, you will place all of your functions into .py files using visual-studio code. 